### 기본 세팅
작동 시간이 너무 빠르면 자동툴로 인식하는 경우가 있어 계속 sleep을 걸음

In [1]:
import os
import time
import pandas as pd

from selenium import webdriver
import pyperclip
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

In [2]:
# 셀레니움 드라이버로 웹 실행
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(3)

In [3]:
# 클립보드에 input을 복사하고, ActionChain을 이용해 복사 내용을 로그인 폼에 붙여넣기
def copy_input(xpath, input):
    pyperclip.copy(input)
    driver.find_element_by_xpath(xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)

In [4]:
# 네이버 id & pw 입력
naver_id = 'bokju128'
naver_pw = 'uks01uks28!'

In [5]:
# 네이버 로그인 
driver.get('https://nid.naver.com/nidlogin.login')             
time.sleep(1)

copy_input('//*[@id="id"]', naver_id)
time.sleep(1)
copy_input('//*[@id="pw"]', naver_pw)
time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

### 주소 및 닉네임 크롤링 테스트
INFP & ENFP 게시판: menuid=18

In [6]:
page = 0
address = []
nickname = []

while page <= 1:
    try:
        # 페이지 및 iframe 전환
        page += 1
        driver.get(f"https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&search.menuid=18&search.boardtype=L&search.page={page}")
        driver.switch_to.frame("cafe_main")
        
        # 더이상 게시글이 없을 경우
        try:
            if driver.find_element_by_class_name("nodata").text == '등록된 게시글이 없습니다.':
                break
        except:
            pass
        
        # 페이지당 주소 및 닉네임 20개 크롤링
        for i in range(1, 31, 1):
            address.append(driver.find_element_by_xpath(f"/html/body/div[1]/div/div[4]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]").get_attribute("href"))
            nickname.append(driver.find_element_by_xpath(f"/html/body/div[1]/div/div[4]/table/tbody/tr[{i}]/td[2]/div/table/tbody/tr/td/a").get_attribute("onclick"))
                            
        # 기존 웹페이지로 복귀
        driver.switch_to.default_content()   
        
    except:
        pass

In [7]:
# 크롤링한 게시글 주소
address

['https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=505279&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=505267&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=505234&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=505233&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=505173&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=505150&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=505147&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype

In [8]:
len(address)

30

In [9]:
# 크롤링한 닉네임이 포함되어 있는 onclick 타입
nickname

["ui(event, 'dhdkcj',3,'enfp성공','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'slimshady0306juw',3,'마더뻑킹인티제','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'koe1012',3,'INFP흰여우','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'koe1012',3,'INFP흰여우','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'yey_',3,'조각INFP','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, '010kl',3,'INFP임둥','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'gydbs1234',3,'INFJ 보라도리','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'kokkkc8k',3,'adorabled isfp','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'astros29',3,'뽀로','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); 

In [10]:
len(nickname)

30

In [11]:
nickname[0]

"ui(event, 'dhdkcj',3,'enfp성공','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;"

In [12]:
type(nickname[0])

str

### onclick 타입 nickname에서 mbti 추출

In [13]:
mbti_group = ['intj', 'intp', 'entj', 'entp',
              'infj', 'infp', 'enfj', 'enfp',
              'istj', 'isfj', 'estj', 'esfj',
              'istp', 'isfp', 'estp', 'esfp']
len(mbti_group)

16

In [14]:
# onclick 타입 nickname 소문자화
for i in range(len(nickname)):
    nickname[i] = nickname[i].lower()

In [15]:
drop_idx = []   

for i in range(len(nickname)):
    check_cnt = 0
    
    for mbti in mbti_group:        
        # 닉네임에 mbti가 포함된 경우 닉네임을 mbti화
        if mbti in nickname[i]:
            nickname[i] = mbti
        
        # 닉네임에 mbti가 포함되지 않은 경우
        if mbti not in nickname[i]:
            check_cnt += 1
        
    # 닉네임에 mbti가 아예 없을 경우 해당 인덱스 저장
    if check_cnt == 16:
        drop_idx.append(i)

In [16]:
nickname

['enfp',
 "ui(event, 'slimshady0306juw',3,'마더뻑킹인티제','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'infp',
 'infp',
 'infp',
 'infp',
 'infj',
 'isfp',
 "ui(event, 'astros29',3,'뽀로','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'dlwlsgur1121',3,'에브','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'infp',
 "ui(event, 'kossh_2001',3,'랑듀엔쁩','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'infp',
 "ui(event, 'dusen3505',3,'걱정하지말고꾜','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'flqls0226',3,'loppoo','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'estp',
 'isfj',
 "ui(event, 'hee5717',3,'엔팁v','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'astros29',3,'뽀로','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(

In [17]:
drop_idx   # 닉네임에 mbti가 포함되지 않아 drop할 인덱스

[1, 8, 9, 11, 13, 14, 17, 18, 19, 20, 21, 25, 27]

In [18]:
len(drop_idx)

13

### 기존 address, nickname에서 drop_idx에 해당되면 드랍

In [19]:
print(' drop전 address 개수: {0}'.format( len(address) ))
print(' drop전 nickname 개수: {0}'.format( len(nickname) ))

 drop전 address 개수: 30
 drop전 nickname 개수: 30


In [20]:
for i in range(len(address)):
    
    while i in drop_idx:
        address.pop(i)
        nickname.pop(i)
        drop_idx.remove(i)
        
        for idx in range(len(drop_idx)):
            drop_idx[idx] -= 1
    
print(' drop후 address 개수: {0}'.format( len(address) ))
print(' drop후 nickname 개수: {0}'.format( len(nickname) ))

 drop후 address 개수: 17
 drop후 nickname 개수: 17


In [21]:
nickname

['enfp',
 'infp',
 'infp',
 'infp',
 'infp',
 'infj',
 'isfp',
 'infp',
 'infp',
 'estp',
 'isfj',
 'entp',
 'infp',
 'esfp',
 'infp',
 'enfp',
 'infp']

### address, nickname으로 게시글 제목과 본문 추출

In [22]:
title = []
content = []

for url in address:
    # 페이지 및 iframe 전환
    driver.get(url)
    driver.switch_to.frame("cafe_main")

    # 제목과 본문 크롤링
    cur_title = driver.find_element_by_css_selector("h3.title_text").text
    cur_content = driver.find_element_by_class_name("se-main-container").text
        
    title.append(cur_title)
    content.append(cur_content)

In [23]:
# 크롤링한 게시글 제목
title

['일상 살면서 필요한 창의적인거 있어나요?요새너무 더우니까 뭐 어떤거 필요하거나 이런거요',
 '저녁 못먹었어요.',
 '앗!여우의 신무기에양!',
 '구름 심리테스트(인척)',
 '인프피 여러분 만남과 이별에 무뎌지는 법은 뭘까요??',
 '엔프피가 사람을 관찰하나요?',
 '엔프피 여자 이별후',
 '인프피 질문해주세요!!',
 '.',
 '인프피분들 한 번 친구면 계속 친구인가요?',
 'ENFP에게 적극적인 이성은 어떤가요??',
 '엔프피 질문드립니다',
 '썸기간 연락문제ㅠㅠㅜㅠ',
 '엔프피 여자 질문..',
 '내 키보드 고장났댜아아아~입미댜아아...',
 '인프피가 장난 많이쳐요?ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 'infp 분들~~']

In [24]:
# 크롤링한 게시글 본문
content

['의견 부탁드려용',
 '이유:피자 먹은게 계속 느끼한 느낌이 들어서,\n저녁 도저히 못먹겠어요.\n가스 활명수랑 파프리카랑 계란 두개로 끝났어요.\n(x파x스......맛도없고..)\n역시 광고는 믿을게 못됩니다.\n아이구 진짜 속터져..엉엉..ㅠㅠ',
 '네...\n결국 as의 복잡한 방식으로 인해..\n여러 키가 눌리지 않아,\n얼리버드 알람의 따라쓰기 문구를 해제할수가 없어서..\n(화면 타이핑으로 하는건 오타가 자주나서..)\n이참에 신무기 하나 들여왔습니다.\n(파일 제목:앗!신무기에양!.jpg)\n이친굽니다.\n아까 시켜서 내일 오긴 옵니다.\n이번만큼은 튼튼하다고 하다는거 as 센터 믿고 샀어요 ㅇㅅㅇ',
 '구름에서 무엇이 보이시나요?\n1. 물고기\n당신은 귀여운것을 좋아하는 사람입니다.\n보자마자 물고기가 떠올랐을 수록 그 사람도 귀여울 확률이 높습니다😌\n2. 새우튀김\n당신은 높은 확률로 다이어트를 시도해 본적이 있거나\n다이어트를 하고 있습니다.\n더운 여름날 과도한 다이어트는 몸을 해칠수도 있으니\n수분 섭취를 충분히 해주시고 충분한 휴식을 취해주세요😉 관리하는 당신이 멋있습니다🤗\n3. 문어\n문어를 찾은 당신.\n당신은 정말 엉뚱하군요. 그 엉뚱함을 제가 칭찬해드리고 싶습니다. 더운 여름날 엉뚱함 +1 정도 추가된다면\n너무 재미있기 때문이죠🥺\n또 다른 엉뚱함을 가지고 계신 분이 있다면 보여줘용~~!🥺✨✨✨✨✨',
 '제가 아직 철이 덜들어서 그런 건지 성숙하지 못한건지\n새로운 만남은 아무생각 없이 시작되어서 너무 정이 금방들고 이별의 순간이 찾아오면 괴롭고 힘들어요\n상대가 날 지치고 질리게 만들지라도요..\n정이 진짜 무서운 것 같아요\n그리고 그 순간 좋았던 추억들도요..\n통달하신 인프피가 계신가요..??',
 '평소에 표정 변화가 그닥 없는 인프제에요.\n예외적으로 몇가지 있는 분노버튼이 눌리면 표정 숨기지 못하는(않는) 편이라는걸\n스스로는 잘 알고있긴한데..;;\n엔프피 썸남이 "원래 표정 잘 못숨기잖아"라고 해

In [25]:
print(len(title), len(content))

17 17


### 데이터프레임화해서 파일로 저장

In [26]:
data = pd.DataFrame({'mbti': nickname, 'title': title, 'content': content})
data

,mbti,title,content
0,enfp,일상 살면서 필요한 창의적인거 있어나요?요새너무 더우니까 뭐 어떤거 필요하거나 이런거요,의견 부탁드려용
1,infp,저녁 못먹었어요.,"이유:피자 먹은게 계속 느끼한 느낌이 들어서,\n저녁 도저히 못먹겠어요.\n가스 활..."
2,infp,앗!여우의 신무기에양!,"네...\n결국 as의 복잡한 방식으로 인해..\n여러 키가 눌리지 않아,\n얼리버..."
3,infp,구름 심리테스트(인척),구름에서 무엇이 보이시나요?\n1. 물고기\n당신은 귀여운것을 좋아하는 사람입니다....
4,infp,인프피 여러분 만남과 이별에 무뎌지는 법은 뭘까요??,제가 아직 철이 덜들어서 그런 건지 성숙하지 못한건지\n새로운 만남은 아무생각 없이...
5,infj,엔프피가 사람을 관찰하나요?,평소에 표정 변화가 그닥 없는 인프제에요.\n예외적으로 몇가지 있는 분노버튼이 눌리...
6,isfp,엔프피 여자 이별후,연락 다시 해 보려고 하는데\n안 잡히겠죠? ㅠ 제가 차인 상황이고\n연락 안 한지...
7,infp,인프피 질문해주세요!!,날이 정말 더워서 냉방병 걸리기도 좋네요... 하루 내내 비실비실 피로함... 다들...
8,infp,.,핸드드립 한잔했습니다.\n하......\n어제 블투키보드가 고장나버린거..\n결국 ...
9,estp,인프피분들 한 번 친구면 계속 친구인가요?,"친구관계인 인프피가 너무 맘에 드는데, 한 번 친구라는 생각이 들면 연인관계로는 잘..."


In [27]:
data.to_csv('INFP & ENFP 게시판 크롤링 데이터(test).csv')